In [182]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

src_path = Path("NZ_Admin_JOBS.xlsx")  
df = pd.read_excel(src_path)
print("\nRaw data size:",df.shape)
df.iloc[:, :6].head(10)


Raw data size: (2708, 6)


,字段1,字段1_link,字段2,字段3,字段4,字段5
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at,Private Advertiser",classification: Administration & Office Suppor...
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at",classification: Administration & Office Suppor...
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,location: AucklandAuckland,"4d ago,at",classification: Administration & Office Suppor...
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,location: SouthlandSouthlandarea: Invercargill...,"1h ago,at",classification: Administration & Office Suppor...
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,location: CanterburyCanterburyarea: Christchur...,"4d ago,at,Private Advertiser",classification: Administration & Office Suppor...
5,Support Officer,https://www.seek.co.nz/job/50640393?type=stand...,Ministry for Primary Industries,location: NorthlandNorthlandarea: WhangareiWha...,"9m ago,at",classification: Administration & Office Suppor...
6,Support Officer,https://www.seek.co.nz/job/50615674?type=stand...,"Ministry of Business, Innovation and Employment",location: WellingtonWellingtonarea: Wellington...,"5d ago,at",classification: Administration & Office Suppor...
7,office administrator,https://www.seek.co.nz/job/50640166?type=stand...,Hepburn Electrical Ltd,location: Bay of PlentyBay of Plentyarea: Roto...,"7m ago,at",classification: Administration & Office Suppor...
8,Office Administrator,https://www.seek.co.nz/job/50639248?type=stand...,Webster Holland Ltd,location: Bay of PlentyBay of Plentyarea: Taur...,"2h ago,at",classification: Administration & Office Suppor...
9,Administration Officer,https://www.seek.co.nz/job/50629393?type=stand...,New Zealand Police,location: CanterburyCanterbury,"3d ago,at",classification: Administration & Office Suppor...


In [183]:
df = df.rename(columns={
    '字段1': 'Job_title',
    '字段1_link': 'Link', 
    '字段2': 'Company_name', 
    '字段3': 'Location&Area',
    '字段4': 'Date',
    '字段5': 'Classification&Subclassification'
})


df.iloc[:, :5].head(10)

,Job_title,Link,Company_name,Location&Area,Date
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at,Private Advertiser"
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at"
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,location: AucklandAuckland,"4d ago,at"
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,location: SouthlandSouthlandarea: Invercargill...,"1h ago,at"
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,location: CanterburyCanterburyarea: Christchur...,"4d ago,at,Private Advertiser"
5,Support Officer,https://www.seek.co.nz/job/50640393?type=stand...,Ministry for Primary Industries,location: NorthlandNorthlandarea: WhangareiWha...,"9m ago,at"
6,Support Officer,https://www.seek.co.nz/job/50615674?type=stand...,"Ministry of Business, Innovation and Employment",location: WellingtonWellingtonarea: Wellington...,"5d ago,at"
7,office administrator,https://www.seek.co.nz/job/50640166?type=stand...,Hepburn Electrical Ltd,location: Bay of PlentyBay of Plentyarea: Roto...,"7m ago,at"
8,Office Administrator,https://www.seek.co.nz/job/50639248?type=stand...,Webster Holland Ltd,location: Bay of PlentyBay of Plentyarea: Taur...,"2h ago,at"
9,Administration Officer,https://www.seek.co.nz/job/50629393?type=stand...,New Zealand Police,location: CanterburyCanterbury,"3d ago,at"


In [184]:
text_cols = df.select_dtypes(include=['object']).columns.tolist()

AMOUNT = r'(?:NZD|\$)\s*\d[\d,]*(?:\.\d+)?\s*[kK]?|\d[\d,]*(?:\.\d+)?\s*[kK]\b'

range_pat  = re.compile(
    rf'({AMOUNT})\s*(?:–|—|-|to|~)\s*({AMOUNT})'
    r'(?:\s*(?:per\s*(?:hour|day|week|month|annum|year)|/(?:hr|h|day|wk|mo|yr)|ph|hourly|annual|annum|p\.?\s*a\.?))?',
    re.I
)
single_pat = re.compile(
    rf'({AMOUNT})'
    r'(?:\s*(?:per\s*(?:hour|day|week|month|annum|year)|/(?:hr|h|day|wk|mo|yr)|ph|hourly|annual|annum|p\.?\s*a\.?))?',
    re.I
)

def extract_salary_bits(text) -> list:
    if pd.isna(text) or str(text).strip()=="":
        return []
    t = str(text)
    hits = [m.group(0).strip(' ,;') for m in range_pat.finditer(t)]
    for m in single_pat.finditer(t):
        frag = m.group(0).strip(' ,;')
        if not any(frag in h for h in hits):
            hits.append(frag)

    seen, out = set(), []
    for h in hits:
        k = h.lower()
        if k not in seen:
            seen.add(k); out.append(h)
    return out

def row_salary(row: pd.Series) -> str:
    seen, out = set(), []
    for c in text_cols:
        val = row[c]
        if pd.isna(val): 
            continue
        for h in extract_salary_bits(val):
            k = h.lower()
            if k not in seen:
                seen.add(k); out.append(h)
    return " | ".join(out)

df["Salary"] = df.apply(row_salary, axis=1)


def remove_salary_bits(text):
    if pd.isna(text) or str(text).strip()=="":
        return text
    t = str(text)
    t = re.sub(range_pat,  "", t)
    t = re.sub(single_pat, "", t)
    t = re.sub(r'\s*[|,;]\s*', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip(' -/ ').strip()
    return t

for colname in ['Location&Area', 'Classification&Subclassification']:
    if colname in df.columns:
        df[colname] = df[colname].apply(remove_salary_bits)


show_cols = [c for c in ['Location&Area', 'Classification&Subclassification', 'Salary'] if c in df.columns]
display(df[show_cols].head(50))


,Location&Area,Classification&Subclassification,Salary
0,location: Bay of PlentyBay of Plentyarea: Taur...,classification: Administration & Office Suppor...,
1,location: Bay of PlentyBay of Plentyarea: Taur...,classification: Administration & Office Suppor...,
2,location: AucklandAuckland,classification: Administration & Office Suppor...,
3,location: SouthlandSouthlandarea: Invercargill...,classification: Administration & Office Suppor...,
4,location: CanterburyCanterburyarea: Christchur...,classification: Administration & Office Suppor...,
5,location: NorthlandNorthlandarea: WhangareiWha...,classification: Administration & Office Suppor...,
6,location: WellingtonWellingtonarea: Wellington...,classification: Administration & Office Suppor...,
7,location: Bay of PlentyBay of Plentyarea: Roto...,classification: Administration & Office Suppor...,
8,location: Bay of PlentyBay of Plentyarea: Taur...,classification: Administration & Office Suppor...,
9,location: CanterburyCanterbury,classification: Administration & Office Suppor...,


In [185]:
def _norm_space(s: str) -> str:
    s = str(s).replace('\u00A0',' ').strip()
    s = re.sub(r'\s+', ' ', s)
    return s

def _collapse_block(s: str) -> str:
    s = _norm_space(s)
    s = re.sub(r'\b([\w\-]+)(\s+\1\b)+', r'\1', s, flags=re.I)
    m = re.fullmatch(r'(.+?)\1+', s)
    if m:
        s = m.group(1).strip()
    return s

def _dedup_parts(s: str) -> str:
    parts = [p.strip() for p in re.split(r'\s*(?:,|/|\||;|•|·)\s*', s) if p and p.strip()]
    seen, out = set(), []
    for p in parts:
        key = p.lower()
        if key not in seen:
            seen.add(key); out.append(p)
    return ', '.join(out) if out else s

# ——Location&Area——
def dedup_location_area(val):
    if pd.isna(val) or str(val).strip()=="":
        return val
    t = str(val)
    t = re.sub(r'\s*location\s*:\s*', 'location: ', t, flags=re.I)
    t = re.sub(r'\s*area\s*:\s*',     ' area: ', t, flags=re.I)

    m_loc  = re.search(r'location:\s*(.*?)(?:\s+area:|$)', t, flags=re.I)
    m_area = re.search(r'area:\s*(.*)$', t, flags=re.I)
    loc  = m_loc.group(1) if m_loc else ''
    area = m_area.group(1) if m_area else ''

    loc  = _dedup_parts(_collapse_block(loc))
    area = _dedup_parts(_collapse_block(area))

    parts = []
    if loc:  parts.append(f"location: {_norm_space(loc)}")
    if area: parts.append(f"area: {_norm_space(area)}")
    return ' '.join(parts)

# ——Classification&Subclassification——
def dedup_class_sub(val):
    if pd.isna(val) or str(val).strip()=="":
        return val
    t = str(val)
    t = re.sub(r'\s*sub\s*classification\s*:\s*', ' subClassification: ', t, flags=re.I)

    m = re.search(r'^(.*?)\s+subClassification:\s*(.*)$', t, flags=re.I)
    main, sub = (m.group(1), m.group(2)) if m else (t, '')

    main = re.sub(r'^\s*(classification|classficition|classfication)\s*:\s*', '', main, flags=re.I)

    main = _dedup_parts(_collapse_block(main))
    sub  = _dedup_parts(_collapse_block(sub))

    main = _norm_space(main)
    sub  = _norm_space(sub)
    return f"{main} subClassification: {sub}" if sub else main


if 'Location&Area' in df.columns:
    df['Location&Area'] = df['Location&Area'].apply(dedup_location_area)

if 'Classification&Subclassification' in df.columns:
    df['Classification&Subclassification'] = df['Classification&Subclassification'].apply(dedup_class_sub)


display(df[[c for c in ['Location&Area','Classification&Subclassification'] if c in df.columns]].head(20))



,Location&Area,Classification&Subclassification
0,location: Bay of Plenty area: Tauranga,Administration & Office Support subClassificat...
1,location: Bay of Plenty area: Tauranga,Administration & Office Support subClassificat...
2,location: Auckland,Administration & Office Support subClassificat...
3,location: Southland area: Invercargill,Administration & Office Support subClassificat...
4,location: Canterbury area: Christchurch,Administration & Office Support subClassificat...
5,location: Northland area: Whangarei,Administration & Office Support subClassificat...
6,location: Wellington area: Wellington Central,Administration & Office Support subClassificat...
7,location: Bay of Plenty area: Rotorua,Administration & Office Support subClassificat...
8,location: Bay of Plenty area: Tauranga,Administration & Office Support subClassificat...
9,location: Canterbury,Administration & Office Support subClassificat...


In [186]:
preferred = ['Job_title', 'Link', 'Company_name', 'Location&Area', 'Date', 'Classification&Subclassification', 'Salary']
cols = [c for c in preferred if c in df.columns] + [c for c in df.columns if c not in preferred]

df_out = df[cols]
df_out.to_csv('NZ_Admin_JOBS_final.csv', index=False, encoding='utf-8-sig', na_rep='')

print('Saved: NZ_Admin_JOBS_final.csv', df_out.shape)


Saved: NZ_Admin_JOBS_final.csv (2708, 7)
